In [1]:
import os
import numpy as np
from datetime import datetime
os.chdir("../")
from datasets import load_dataset, Dataset, DatasetDict
from src.data import DataProcessing, HF_Dataset
from src.utils import get_split_limit

/opt/mamba/lib/python3.10/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [2]:
dataset = load_dataset("monash_tsf", "traffic_hourly")
freq = "1H"

Found cached dataset monash_tsf (/home/onyxia/.cache/huggingface/datasets/monash_tsf/traffic_hourly/1.0.0/82998723d55d6edbc664cfaebe371004d849846ee5f61b57f1f974000d44c050)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
data_pipeline = DataProcessing(dataset)
# mtrain, mtest = data_pipeline.multi_variate_format(freq)

In [4]:
get_split_limit(10*[1., 2., 3.], np.array([0.7, 0.2, 0.1]))

array([21, 26, 30])

In [5]:
ds = HF_Dataset(
    [datetime(2023, 4, 12, 12, 0, 0), datetime(2023, 4, 12, 13, 0, 0)], 
    [10*[1, 2, 3], 10*[4, 5, 6]],
    np.array([0.7, 0.2, 0.1]),
    [10*[0, 0, 0], 10*[0, 0, 0]],
    [10*[None, None, None], 10*[None, None, None]]).getDatasetDict()

In [29]:
data_pipeline = DataProcessing(ds)
data_pipeline.multi_variate_format(freq)

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 30 and the array at index 1 has size 31

In [21]:
from gluonts.dataset.multivariate_grouper import (
    MultivariateGrouper
)

num_of_variates = len(ds["train"])

In [22]:
ds["train"][0]

{'start': Period('2023-04-12 12:00', 'H'),
 'target': [1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3],
 'feat_static_cat': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'feat_dynamic_real': [None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 'item_id': 'T0'}

In [23]:
train_grouper = MultivariateGrouper(
                            max_target_dim=num_of_variates
                        )

In [28]:
train_grouper(ds["test"])[0]["target"]

array([[1., 2., 3., 1., 2., 3., 1., 2., 3., 1., 2., 3., 1., 2., 3., 1.,
        2., 3., 1., 2., 3., 1., 2., 3., 1., 2., 3., 1., 2., 3., 2.],
       [5., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6.,
        4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6.]],
      dtype=float32)